In [4]:
import os
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import col

conf = SparkConf().setAppName('PySparkShell').setMaster('local')
sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)

hostname = "127.0.0.1" 
dbname = "spark_test"
jdbcPort = 3306
username = "root"
password = "rootpwd"
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(hostname,jdbcPort, dbname,username,password)


query = "window_test"
df1 = sqlContext.read.format('jdbc').options(driver = 'com.mysql.jdbc.Driver',url=jdbc_url, dbtable=query ).load()
df1.show()

+---+-----+-------------------+-----+-----------+
| id| name|           end_time|value|    comment|
+---+-----+-------------------+-----+-----------+
|  1|node1|2017-03-24 08:30:00|    5|notrelevant|
|  2|node1|2017-03-24 09:00:00|    3|notrelevant|
|  3|node1|2017-03-24 09:30:00|    8|notrelevant|
|  4|node2|2017-03-24 10:00:00|    5|notrelevant|
|  5|node2|2017-03-24 10:30:00|    3|notrelevant|
|  6|node2|2017-03-24 11:00:00|    1|notrelevant|
|  7|node2|2017-03-24 11:30:00|    3|notrelevant|
|  8|node2|2017-03-24 12:00:00|    5|notrelevant|
+---+-----+-------------------+-----+-----------+



In [5]:
from pyspark.sql.functions import col, window, max

df1.groupBy(
    col("name"), window(col("end_time"), "2 hour", "20 minute")
)\
.agg(max("value"))\
.collect()
#.filter(col("max(value)") < 6)\
#.select("name", "max(value)").distinct().show()

#.agg(max("value")).withRenamedColumn("max('value')", "2_hour_max_value")).filter(col("2_hour_max_value") < 6).select("name").distinct()

[Row(name='node2', window=Row(start=datetime.datetime(2017, 3, 24, 9, 20), end=datetime.datetime(2017, 3, 24, 11, 20)), max(value)=5),
 Row(name='node2', window=Row(start=datetime.datetime(2017, 3, 24, 10, 40), end=datetime.datetime(2017, 3, 24, 12, 40)), max(value)=5),
 Row(name='node1', window=Row(start=datetime.datetime(2017, 3, 24, 9, 0), end=datetime.datetime(2017, 3, 24, 11, 0)), max(value)=8),
 Row(name='node1', window=Row(start=datetime.datetime(2017, 3, 24, 7, 0), end=datetime.datetime(2017, 3, 24, 9, 0)), max(value)=5),
 Row(name='node2', window=Row(start=datetime.datetime(2017, 3, 24, 8, 40), end=datetime.datetime(2017, 3, 24, 10, 40)), max(value)=5),
 Row(name='node1', window=Row(start=datetime.datetime(2017, 3, 24, 7, 20), end=datetime.datetime(2017, 3, 24, 9, 20)), max(value)=5),
 Row(name='node1', window=Row(start=datetime.datetime(2017, 3, 24, 8, 20), end=datetime.datetime(2017, 3, 24, 10, 20)), max(value)=8),
 Row(name='node2', window=Row(start=datetime.datetime(2017, 